# Quick Start - Run local code as SageMaker training job


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

---


We're introducing a new capability in AWS SageMaker Python SDK that allows data scientists to run their Python functions as SageMaker Jobs to take advantage of the compute power offered by SageMaker. This sample notebook is a quick introduction to this new capability with dummy Python functions.

For more details about the feature, please checkout the AWS Developer Guide [here](https://docs.aws.amazon.com/sagemaker/latest/dg/train-remote-decorator.html)


## Install the dependencies

In [ ]:
%pip install -r ./requirements.txt

## Setup Configuration file path
We need set the directory in which the config.yaml file resides so that remote decorator can make use of the settings.

In [ ]:
import os

# Set path to config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

## Invoke Python function as a SageMaker job

There are two ways users could invoke a function as a job.

* Using the decorator. When the decorated function is invoked, it executes synchronously

In [ ]:
from sagemaker.remote_function import remote


@remote
def divide(x, y):
    print(f"Calculating {x}/{y}")
    return x / y


divide(3, 2)

* Using the RemoteExecutor APIs. They follow the pattern of the Python [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) APIs.

In [ ]:
def divide(x, y):
    print(f"Calculating {x}/{y}")
    return x / y


from sagemaker.remote_function import RemoteExecutor

with RemoteExecutor(max_parallel_jobs=1, keep_alive_period_in_seconds=30) as executor:
    futures = [executor.submit(divide, x, 2) for x in [1, 2, 3]]

In [ ]:
[future.result() for future in futures]

## Customize the decorator settings

In the examples above, the @remote decorator and RemoteExecutor looked for configurations in the path set in the environment variable `SAGEMAKER_USER_CONFIG_OVERRIDE`/
We set the value to use the local configuration file `config.yaml` present in the current directory `./`

You can override the configurations by specifying the decorator arguments directly. In the following example, instead of launching the job with `ml.m5.xlarge`,
as specified in the `./config.yaml`, run the function with a more powerful instance

In [ ]:
@remote(instance_type="ml.m5.4xlarge")
def divide(x, y):
    print(f"Calculating {x}/{y}")
    return x / y


divide(3, 2)

## Add extra dependencies using conda environment yml file

In the example below, the function will run in a new conda environment where pandas and sagemaker will be installed.
(Note that the cell will not run if executed in SageMaker Studio.)

In [ ]:
import pandas as pd


@remote(dependencies="./environment.yml")
def multiply(dataframe: pd.DataFrame, factor: float):
    return dataframe * factor


df = pd.DataFrame(
    {
        "A": [14, 4, 5, 4, 1],
        "B": [5, 2, 54, 3, 2],
        "C": [20, 20, 7, 3, 8],
        "D": [14, 3, 6, 2, 6],
    }
)

multiply(df, 10.0)

## Common errors

### SerializationError and DeserializationError

Behind the scenes, the function and function arguments and returns are serialized and deserialized using Pickle.

While SerializationError occurs when passing an unpickle-able object as function argument, such as XGBoost DMatrix, open file object. DeserializationError typically happens when there is discrepancy between the dependencies in the local environments and dependencies in
the job environments. In the following example, the latest pandas is used to declare a dataframe. The dataframe is passed to the function
call. On the job side, an older version of pandas is installed. The dataframe can't be deserialized.

In [ ]:
import pandas as pd


@remote(dependencies="./incompatible_requirements.txt")
def multiply(dataframe: pd.DataFrame, factor: float):
    return dataframe * factor


df = pd.DataFrame(
    {
        "A": [14, 4, 5, 4, 1],
        "B": [5, 2, 54, 3, 2],
        "C": [20, 20, 7, 3, 8],
        "D": [14, 3, 6, 2, 6],
    }
)

multiply(df, 10.0)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-remote-function|quick_start|quick_start.ipynb)
